## Arguana

In [1]:
from kdir_src.fire.promptor_fire import Promptor_fire

In [2]:
promptot = Promptor_fire('arguana')

In [3]:
from kdir_src.dataset.beir import get_sentences_queries
sentences, queries_ids = get_sentences_queries('arguana')

/local1/ronaldinho/enviroments/env_kdir/lib/python3.10/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/8674 [00:00<?, ?it/s]

Procesando queries: 100%|█████████████████████████████████████████████████████████████| 1406/1406 [00:00<00:00, 4361827.98it/s]


In [4]:
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
encoder_models = load_models()
c_emb_list = get_query_embeddings(encoder_models, [sentences[3]],1,True)

Loading contriever...
Loading contriever FT...
Loading question dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRCon

Loading context dpr...


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading bge large...
Loading gte large...
Loading bm25...


Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 115.04batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [5]:
from kdir_src.utils.qdrant import recuperar_documentos
collection_name= f"kdir_arguana"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)

In [7]:
prompts = promptot.get_prompt(sentences[3], rel_docs)

In [11]:
from kdir_src.generators.mistral_ai import MistralGenerator
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

In [8]:
print(prompts[0])


Task: Write a compelling and well-founded counter-argument for the following argument.

- Use the 'Reference Arguments' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
- **Closely match the style of the 'Reference Arguments'. This includes their approximate length (e.g., a single concise sentence), tone, and overall structure.**
- The Counter-Argument should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final Counter-Argument, with no introduction or explanation.

Argument: Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating underc

In [13]:
ans1 = generator.generate(prompts[0])
print(ans1[0])

While it is true that certain types of food poisoning are associated with meat and dairy products, it is essential to recognize that foodborne illnesses can also originate from plant-based sources. For instance, vegetables and fruits can harbor bacteria like Listeria monocytogenes and E. coli, especially if they are not properly washed or handled. Moreover, zoonotic diseases, which can be transmitted from animals to humans, are not exclusively linked to livestock; many pathogens originate from wildlife, making the risk of transmission present regardless of dietary choices. Additionally, the safety of food is not solely determined by its source but by the hygiene and safety standards in place during production, processing, and preparation. Therefore, focusing on stringent food safety practices across all food types is crucial, rather than advocating for a specific diet to mitigate food poisoning risks.


In [9]:
print(prompts[1])


Task: Generate a new, insightful Argument using the reference arguments for context.

- Use the 'Reference Arguments' provided below for context and inspiration.
- The new query should be logical, well-formed, and directly related to the original inputs.
- **Your response must ONLY be the text of the final Generated Argument, with no introduction or explanation.**

Original Argument: Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating undercooked beef or drinking unpasteurised milk. [1]  Close contact between humans and animals also leads to zoonosis – diseases such as bird ‘flu which can be passed on from animals 

In [14]:
ans2 = generator.generate(prompts[1])
print(ans2[0])

While it is true that adopting a vegetarian diet can reduce the risk of certain types of food poisoning, it is essential to recognize that food safety extends beyond the choice between meat and plant-based diets. The primary concern should be the overall hygiene and safety standards in food production and preparation, regardless of the type of food. For instance, vegetables can also be sources of foodborne illnesses, such as Listeria monocytogenes, which can be transmitted through raw vegetables. Therefore, the focus should be on ensuring that all food, whether animal or plant-based, is produced and handled in a manner that minimizes the risk of contamination. Additionally, the ethical and environmental implications of food choices must be considered. For example, the culling of badgers to prevent bovine TB highlights the complex interplay between animal welfare, public health, and environmental sustainability. Thus, a balanced approach that prioritizes food safety, ethical considerati

In [10]:
print(prompts[2])


Task: Extract the core claim and the key premises or evidence used to support it from the Argument

- Use the 'Reference Arguments' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
- The Claim should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final Claim, with no introduction or explanation.

Argument: Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating undercooked beef or drinking unpasteurised milk. [1]  Close contact between humans and animals also leads to zoonosis – diseases such as bird ‘flu which can be passed on from anim

In [15]:
ans3 = generator.generate(prompts[2])
print(ans3[0])

Adopting a vegetarian diet can significantly reduce the risk of food poisoning, as the majority of dangerous foodborne pathogens are associated with the consumption of meat and animal products. Key pathogens such as Campylobacter, Salmonella, and E. coli are commonly found in raw or undercooked meat, poultry, and unpasteurized dairy products. Additionally, close contact with animals, which is more prevalent in meat production, increases the risk of zoonotic diseases. Therefore, by eliminating meat and animal products from one's diet, the likelihood of contracting these foodborne illnesses is substantially decreased.


## NFCOrpus

In [1]:
from kdir_src.fire.promptor_fire import Promptor_fire
from kdir_src.dataset.beir import get_sentences_queries
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
from kdir_src.utils.qdrant import recuperar_documentos
from kdir_src.generators.mistral_ai import MistralGenerator

/local1/ronaldinho/enviroments/env_kdir/lib/python3.10/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dataset_name='nfcorpus'
promptot = Promptor_fire(dataset_name)
sentences, queries_ids = get_sentences_queries(dataset_name)
encoder_models = load_models()
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

  0%|          | 0/3633 [00:00<?, ?it/s]

Procesando queries: 100%|███████████████████████████████████████████████████████████████| 323/323 [00:00<00:00, 3058149.42it/s]


Loading contriever...
Loading contriever FT...
Loading question dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRCon

Loading context dpr...


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading bge large...
Loading gte large...
Loading bm25...


In [3]:
c_emb_list = get_query_embeddings(encoder_models, [sentences[3]],1,True)

Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 161.15batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [4]:
from kdir_src.generators.mistral_ai import MistralGenerator

In [64]:
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

In [58]:
PROMPT_TEMPLATE = """
Task: Write a compelling and well-founded counter-argument for the following passage.

- Use the 'Reference Arguments' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
---> - **Closely match the style of the 'Reference Arguments'. This includes their approximate length (e.g., a single concise sentence), tone, and overall structure.**
- The counter-argument should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final counter-argument, with no introduction or explanation.

Passage: {passage}
Reference Arguments: {reference_arguments}
Counter-Argument:
"""


In [59]:
def formatear_doc(arg):
    contraargumento = f"{arg.payload['title']}\n\n{arg.payload['text']}\n\n"
    return contraargumento


def formatear_prompt_argumento(
    passage: str,
    candidate_counter_arguments: list[str]
) -> str:
    """
    Formatea el prompt final para generar un contraargumento,
    incluyendo un pasaje y una lista de candidatos a evaluar.

    Args:
        passage (str): El texto principal para el cual se necesita un contraargumento.
        candidate_counter_arguments (list[str]): Una lista de "pseudo-contraargumentos"
                                                   para ser evaluados por el modelo.

    Returns:
        str: El prompt de texto completo, formateado y listo para ser enviado a un modelo de lenguaje.
    """
    # Formatea la lista de candidatos para que sea legible en el prompt.
    if candidate_counter_arguments:
        # Crea una cadena de texto con los argumentos numerados.
        formatted_candidates = "\n".join(
            [f"{i+1}. {formatear_doc(arg)}" for i, arg in enumerate(candidate_counter_arguments)]
        )
    else:
        # Maneja el caso donde no se proporcionan candidatos.
        formatted_candidates = "None provided."

    # Rellena la plantilla con la información proporcionada.
    final_prompt = PROMPT_TEMPLATE.format(
        passage=passage,
        reference_arguments=formatted_candidates
    )

    return final_prompt

In [7]:
from kdir_src.dataset.beir import get_sentences_queries

sentences, queries_ids = get_sentences_queries('arguana')

/local1/ronaldinho/enviroments/env_kdir/lib/python3.10/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/8674 [00:00<?, ?it/s]

Procesando queries: 100%|█████████████████████████████████████████████████████████████| 1406/1406 [00:00<00:00, 4008967.66it/s]


In [11]:
sentences[3]

'Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating undercooked beef or drinking unpasteurised milk. [1]  Close contact between humans and animals also leads to zoonosis – diseases such as bird ‘flu which can be passed on from animals to humans. Using animal brains in the processed feed for livestock led to BSE in cattle and to CJD in humans who ate beef from infected cows.  [1] Causes of food poisoning, nhs.co.uk, 23rd June 2009'

In [12]:
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
encoder_models = load_models()
c_emb_list = get_query_embeddings(encoder_models, [sentences[3]],1,True)

Loading contriever...
Loading contriever FT...
Loading question dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRCon

Loading context dpr...


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading bge large...
Loading gte large...
Loading bm25...


Procesando dpr...: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.14batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [8]:
from kdir_src.utils.qdrant import recuperar_documentos

In [16]:
collection_name= f"kdir_arguana"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)

In [18]:
rel_docs[0]

ScoredPoint(id=7, version=0, score=1.0, payload={'doc_id': 'test-environment-aeghhgwpe-pro04a', 'title': 'animals environment general health health general weight philosophy ethics', 'text': 'Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating undercooked beef or drinking unpasteurised milk. [1]  Close contact between humans and animals also leads to zoonosis – diseases such as bird ‘flu which can be passed on from animals to humans. Using animal brains in the processed feed for livestock led to BSE in cattle and to CJD in humans who ate beef from infected cows.  [1] Causes of food poisoning, nhs.co.uk, 23rd June 20

In [60]:
formateado=formatear_prompt_argumento(sentences[3],rel_docs)

In [61]:
print(formateado)


Task: Write a compelling and well-founded counter-argument for the following passage.

- Use the 'Reference Arguments' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
---> - **Closely match the style of the 'Reference Arguments'. This includes their approximate length (e.g., a single concise sentence), tone, and overall structure.**
- The counter-argument should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final counter-argument, with no introduction or explanation.

Passage: Being vegetarian reduces risks of food poisoning  Almost all dangerous types of food poisoning are passed on through meat or eggs. So Campylobacter bacteria, the most common cause of food poisoning in England, are usually found in raw meat and poultry, unpasteurised milk and untreated water. Salmonella come from raw meat, poultry and dairy products and most cases of escherichia coli (E-Coli) food poisoning occur after eating und

In [36]:
ans=generator.generate(formateado)

In [37]:
print(ans[0])

### Analysis of the Passage

The passage argues that being vegetarian reduces the risk of food poisoning. It supports this claim by citing specific bacteria and diseases associated with meat and dairy products, such as Campylobacter, Salmonella, and E. coli. The passage also mentions zoonotic diseases like bird flu and the risks associated with BSE and CJD, which are linked to animal-derived products.

### Evaluation of Candidate Counter-Arguments

1. **Candidate Counter-Argument 1**: This argument is incoherent and lacks a clear structure. It does not present a logical counter-argument to the passage.

2. **Candidate Counter-Argument 2**: This argument highlights that food poisoning can occur from sources other than meat, such as raw vegetables contaminated with Listeria. It also mentions that zoonotic diseases can come from wildlife, not just livestock. This argument is relevant and logical but could be more focused and structured.

3. **Candidate Counter-Argument 3**: This argument 

In [62]:
ans2=generator.generate(formateado)
print(ans2[0])

While it is true that certain types of food poisoning are associated with meat and dairy products, the risk can be significantly mitigated through proper food handling, cooking, and regulatory standards. Many cases of food poisoning stem from poor hygiene practices rather than the inherent properties of the food itself. For instance, both meat and plant-based foods can be contaminated with pathogens like Listeria if not handled correctly. Moreover, the emphasis should be on ensuring that all food, regardless of its source, is produced and prepared in a safe and hygienic manner. Additionally, a balanced diet that includes meat can provide essential nutrients that are harder to obtain from a vegetarian diet, such as vitamin B12 and certain types of iron, which are crucial for overall health and productivity.


In [66]:
ans2=generator.generate(formateado)
print(ans2[0])

While it is true that certain types of food poisoning are associated with meat and dairy products, it is not accurate to claim that adopting a vegetarian diet entirely eliminates the risk of foodborne illnesses. Vegetables and plant-based foods can also harbor harmful bacteria such as Listeria monocytogenes, which can cause severe food poisoning. Moreover, the risk of zoonotic diseases is not solely mitigated by avoiding animal products, as many pathogens originate from wildlife and can be transmitted through environmental contact. Therefore, focusing solely on dietary changes may not be the most effective strategy for reducing foodborne illnesses. Instead, comprehensive food safety measures and hygiene practices across all food types are essential for minimizing health risks.


In [2]:
from kdir_src.generators.mistral_ai import MistralGenerator
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

In [18]:
# --- PLANTILLA DE PROMPT (NFCorpus - Síntesis Multi-Documento) ---
PROMPT_TEMPLATE = """
Task: Write a single, new, and comprehensive medical article passage that provides a definitive answer to the user's query.

- Use the 'Source Documents' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
- **Closely match the style of the 'Source Documents'. This includes their approximate length (e.g., a single concise sentence), tone, and overall structure.**
- The Generated Document should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final Generated Document, with no introduction or explanation.

Query: {query}

Source Documents:
{reference_docs}

Generated Document:
"""

In [19]:
def formatear_doc(arg):
    contraargumento = f"{arg.payload['title']}\n\n{arg.payload['text']}\n\n"
    return contraargumento


def formatear_prompt_argumento(
    query: str,
    reference_docs: list
) -> str:
    """
    Formatea el prompt final para generar un contraargumento,
    incluyendo un pasaje y una lista de candidatos a evaluar.

    Args:
        passage (str): El texto principal para el cual se necesita un contraargumento.
        candidate_counter_arguments (list[str]): Una lista de "pseudo-contraargumentos"
                                                   para ser evaluados por el modelo.

    Returns:
        str: El prompt de texto completo, formateado y listo para ser enviado a un modelo de lenguaje.
    """
    # Formatea la lista de candidatos para que sea legible en el prompt.
    if reference_docs:
        # Crea una cadena de texto con los argumentos numerados.
        formatted_candidates = "\n".join(
            [f"{i+1}. {formatear_doc(arg)}" for i, arg in enumerate(reference_docs)]
        )
    else:
        # Maneja el caso donde no se proporcionan candidatos.
        formatted_candidates = "None provided."

    # Rellena la plantilla con la información proporcionada.
    final_prompt = PROMPT_TEMPLATE.format(
        query=query,
        reference_docs=formatted_candidates
    )

    return final_prompt

In [6]:
from kdir_src.dataset.beir import get_sentences_queries
sentences, queries_ids = get_sentences_queries('nfcorpus')

/local1/ronaldinho/enviroments/env_kdir/lib/python3.10/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/3633 [00:00<?, ?it/s]

Procesando queries: 100%|███████████████████████████████████████████████████████████████| 323/323 [00:00<00:00, 2693360.22it/s]


In [7]:
sentences[3]

'What’s Driving America’s Obesity Problem?'

In [8]:
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
encoder_models = load_models()
c_emb_list = get_query_embeddings(encoder_models, [sentences[3]],1,True)

Loading contriever...
Loading contriever FT...
Loading question dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRCon

Loading context dpr...


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading bge large...
Loading gte large...
Loading bm25...


Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 131.03batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [9]:
from kdir_src.utils.qdrant import recuperar_documentos
collection_name= f"kdir_nfcorpus"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)

In [20]:
formateado=formatear_prompt_argumento(sentences[3],rel_docs)
print(formateado)


Task: Write a single, new, and comprehensive medical article passage that provides a definitive answer to the user's query.

- Use the 'Source Documents' provided below as inspiration or incorporate their ideas if they are relevant and helpful.
- **Closely match the style of the 'Source Documents'. This includes their approximate length (e.g., a single concise sentence), tone, and overall structure.**
- The Generated Document should be logical and maintain a respectful tone.
- Your response must ONLY be the text of the final Generated Document, with no introduction or explanation.

Query: What’s Driving America’s Obesity Problem?

Source Documents:
1. Agricultural policies, food and public health

A historical view on how our agricultural systems evolved and how they are contributing to obesity and disease.


2. Tobacco and obesity epidemics: not so different after all?

Short abstract Campaigns to promote healthy eating are undermined by the ubiquity of processed, energy dense foods.

In [21]:
ans=generator.generate(formateado)
print(ans[0])

America's obesity problem is driven by a complex interplay of factors rooted in agricultural policies, food availability, and neurobiological responses to food. Historically, agricultural policies have prioritized the production of calorie-dense, processed foods, which are now ubiquitous in the American diet. These foods, often high in sugar and unhealthy fats, are designed to be highly palatable and rewarding, triggering neurobiological pathways similar to those activated by addictive substances. This creates a cycle of overconsumption and difficulty in resisting unhealthy food choices, akin to addictive behaviors. Moreover, the glycemic index of these foods can exacerbate issues related to obesity, diabetes, and cardiovascular disease, further complicating public health efforts. Campaigns promoting healthy eating are often undermined by the pervasive availability of these processed, energy-dense foods, necessitating a global strategy to address the rising prevalence of obesity. While

In [5]:
collection_name= f"kdir_{dataset_name}"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)
prompts = promptot.get_prompt(sentences[3], rel_docs)

In [6]:
ans1 = generator.generate(prompts[0])
print(ans1[0])

America's obesity problem is driven by a complex interplay of factors rooted in agricultural policies, food availability, and neurobiological responses to food. Historically, agricultural policies have prioritized the production of high-calorie, energy-dense foods, which are now ubiquitous in the American diet. These processed foods, often high in sugar and unhealthy fats, are designed to be highly palatable and readily available, undermining efforts to promote healthy eating. The ubiquity of such foods creates an environment where making healthier choices is challenging, much like the challenges faced in reducing tobacco use. Moreover, recent research suggests that obesity shares similarities with addictive disorders, both in terms of neurobiological mechanisms and cognitive functions. Neuroendocrine and imaging studies indicate a close link between the regulation of appetite and the brain's reward systems, which can lead to a lack of control in resisting consumption. Additionally, th

In [7]:
ans2 = generator.generate(prompts[1])
print(ans2[0])

How do agricultural policies and the prevalence of processed, energy-dense foods contribute to the obesity epidemic in America, and what role do addictive behaviors and the glycemic index play in this context? Additionally, what are the potential benefits and limitations of nutraceutical supplements in addressing obesity?


In [8]:
ans3 = generator.generate(prompts[2])
print(ans3[0])

America's obesity problem is driven by a combination of several key factors:

1. **Agricultural Policies and Food Systems**: Historical agricultural policies have favored the production of cheap, calorie-dense foods, which are often high in sugar and unhealthy fats. These foods are widely available and heavily marketed, making it easier for people to consume more calories than they need.

2. **Processed, Energy-Dense Foods**: The ubiquity of processed, energy-dense foods undermines efforts to promote healthy eating. These foods are designed to be highly palatable and convenient, leading to overconsumption.

3. **Addictive Properties of Food**: Research suggests that obesity shares similarities with addictive disorders. The neurobiological and cognitive mechanisms involved in obesity can make it difficult for individuals to resist consuming high-calorie foods, contributing to a lack of control over eating habits.

4. **Glycemic Index and Dietary Choices**: The glycemic index of foods ca

## FIQA

In [11]:
from kdir_src.fire.promptor_fire import Promptor_fire
from kdir_src.dataset.beir import get_sentences_queries
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
from kdir_src.utils.qdrant import recuperar_documentos
from kdir_src.generators.mistral_ai import MistralGenerator

In [15]:
dataset_name='fiqa'
promptot = Promptor_fire(dataset_name)
sentences, queries_ids = get_sentences_queries(dataset_name)
#encoder_models = load_models()
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

  0%|          | 0/57638 [00:00<?, ?it/s]

Procesando queries: 100%|███████████████████████████████████████████████████████████████| 648/648 [00:00<00:00, 3393144.81it/s]


In [26]:
c_emb_list = get_query_embeddings(encoder_models, [sentences[4]],1,True)
collection_name= f"kdir_{dataset_name}"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)
prompts = promptot.get_prompt(sentences[4], rel_docs)

Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 143.80batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [27]:
ans1 = generator.generate(prompts[0])
print(ans1[0])

When a business closes, managing your 401(k) becomes a critical task. Unlike changing jobs, where rolling over to an Individual Retirement Account (IRA) is often the best remedy, closing a business presents unique challenges. Typically, you cannot roll funds from a 401(k) while still employed with the company, but since the business is closing, you are effectively separated from service, allowing you to initiate a rollover. This process can help you avoid the pitfalls of 401(k) choices, which are often limited and less flexible than IRA options. When you roll over your 401(k) to an IRA, you gain more control over your investments and can potentially benefit from a wider range of investment options. It's important to note that all distributions from a regular (non-Roth) 401(k) are considered regular income and are includible in taxable income at retirement, except for qualified distributions from designated Roth accounts. Therefore, consulting with a financial advisor to understand the 

In [28]:
ans2 = generator.generate(prompts[1])
print(ans2[0])

What are the specific tax implications and rollover options for a 401(k) plan when a business closes, and how do these options compare to the rollover process when changing jobs or retiring, particularly in terms of the timing and tax treatment of distributions?


In [29]:
ans3 = generator.generate(prompts[2])
print(ans3[0])

If your business closes, you can transfer your 401(k) funds to an Individual Retirement Account (IRA) without incurring taxes or penalties, provided you follow the rollover rules. This is often recommended because IRAs typically offer a wider range of investment options compared to 401(k) plans. Alternatively, if you start a new business, you could open a self-employed 401(k) and roll your funds into it, allowing you to contribute up to $50,000 annually, including personal contributions and profit-sharing/matching portions. Ensure you consult with a financial advisor to understand the specific rules and benefits applicable to your situation.


## scidocs

In [31]:
from kdir_src.fire.promptor_fire import Promptor_fire
from kdir_src.dataset.beir import get_sentences_queries
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
from kdir_src.utils.qdrant import recuperar_documentos
from kdir_src.generators.mistral_ai import MistralGenerator

In [32]:
dataset_name='scidocs'
promptot = Promptor_fire(dataset_name)
sentences, queries_ids = get_sentences_queries(dataset_name)
#encoder_models = load_models()
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

  0%|          | 0/25657 [00:00<?, ?it/s]

Procesando queries: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1440351.65it/s]


In [33]:
c_emb_list = get_query_embeddings(encoder_models, [sentences[4]],1,True)
collection_name= f"kdir_{dataset_name}"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)
prompts = promptot.get_prompt(sentences[4], rel_docs)

Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 158.16batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [35]:
ans1 = generator.generate(prompts[0])
print(ans1[0])

Ad hoc retrieval experiments using WordNet and automatically constructed thesauri present a promising avenue for enhancing information retrieval systems. While traditional thesauri have shown inconsistent benefits and are challenging to construct automatically for large text databases, methods like PhraseFinder offer a viable solution by generating collection-dependent association thesauri from full-text document collections. These thesauri can be integrated into systems like INQUERY, which supports natural language queries, to evaluate and compare different types of association thesauri across various collections. Additionally, algorithmic approaches such as symbolic branch-and-bound search and connectionist Hopfield net activation provide efficient means for concept exploration in large, heterogeneous knowledge networks. These algorithms, tested on extensive text-based networks, demonstrate comparable performance to manual browsing but with significantly reduced effort. Concept-based

In [36]:
ans2 = generator.generate(prompts[1])
print(ans2[0])

"Comparative Analysis of Automatic Thesaurus Construction Methods for Enhancing Ad Hoc Retrieval Performance in Large Text Databases"


In [38]:
ans3 = generator.generate(prompts[2])
print(ans3[0])

Ad hoc retrieval experiments using WordNet and automatically constructed thesauri aim to enhance information retrieval systems by leveraging semantic relationships between terms. Traditional keyword-based retrieval methods often fall short due to variations in terminology and the need for comprehensive human knowledge. Concept-based retrieval approaches, such as those using Explicit Semantic Analysis (ESA) and automatically constructed thesauri, address these issues by augmenting text representation with concept-based features. Techniques like PhraseFinder and statistical clustering methods have been developed to automatically generate thesauri from large text collections, improving retrieval effectiveness. Algorithmic approaches, including symbolic branch-and-bound search and neural network-based methods, facilitate efficient traversal of large, heterogeneous knowledge networks. These methods have shown comparable or superior performance to manual browsing, reducing cognitive load and

## scifact

In [1]:
from kdir_src.fire.promptor_fire import Promptor_fire
from kdir_src.dataset.beir import get_sentences_queries
from kdir_src.models.all_models import get_query_embeddings
from kdir_src.models.all_models import load_models
from kdir_src.utils.qdrant import recuperar_documentos
from kdir_src.generators.mistral_ai import MistralGenerator

/local1/ronaldinho/enviroments/env_kdir/lib/python3.10/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dataset_name='scifact'
promptot = Promptor_fire(dataset_name)
sentences, queries_ids = get_sentences_queries(dataset_name)
encoder_models = load_models()
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
generator = MistralGenerator('mistral-small-2503', KEY, 1, 512, 0.0)

  0%|          | 0/5183 [00:00<?, ?it/s]

Procesando queries: 100%|███████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 2953735.21it/s]


Loading contriever...
Loading contriever FT...
Loading question dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRCon

Loading context dpr...


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading bge large...
Loading gte large...
Loading bm25...


In [3]:
c_emb_list = get_query_embeddings(encoder_models, [sentences[4]],1,True)
collection_name= f"kdir_{dataset_name}"
rel_docs = recuperar_documentos(collection_name, c_emb_list[0][0].tolist(), 'contriever', 5)
prompts = promptot.get_prompt(sentences[4], rel_docs)

Procesando dpr...: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 145.10batch/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Procesando BM25 embeddings
Embeddings finalizado


In [4]:
ans1 = generator.generate(prompts[0])
print(ans1[0])

A deficiency of vitamin B12 is well-documented to increase blood levels of homocysteine, a condition known as hyperhomocysteinemia. This relationship is supported by the understanding that vitamin B12, along with folic acid, plays a crucial role in the metabolism of homocysteine. Specifically, vitamin B12 is essential for the conversion of homocysteine to methionine, a process that is impaired in the absence of adequate B12 levels. Consequently, elevated homocysteine levels are often observed in individuals with vitamin B12 deficiency, highlighting the critical interplay between these nutrients in maintaining homocysteine homeostasis. This biochemical interaction underscores the importance of ensuring sufficient vitamin B12 intake to prevent the adverse cardiovascular and vascular outcomes associated with elevated homocysteine levels.


In [5]:
ans2 = generator.generate(prompts[1])
print(ans2[0])

"Does vitamin B12 supplementation independently affect blood homocysteine levels in individuals with normal folic acid intake, and if so, to what extent?"


In [6]:
ans3 = generator.generate(prompts[2])
print(ans3[0])

The claim that "A deficiency of vitamin B12 increases blood levels of homocysteine" is supported by the information provided in the source documents, albeit indirectly. While the documents primarily focus on the role of folic acid in reducing homocysteine levels, they also imply the interconnectedness of B vitamins in homocysteine metabolism.

Document 2 mentions that "Higher doses of folic acid and cyanocobalamin (vitamin B12) supplements may be required in some patients" to manage hyperhomocysteinemia. This suggests that vitamin B12 plays a role in homocysteine metabolism. Document 4 explicitly states that "Folate and vitamin B12 are two vital regulators in the metabolic process of homocysteine." This indicates that a deficiency in vitamin B12, similar to a deficiency in folate, can disrupt the normal metabolism of homocysteine, leading to increased blood levels.

Therefore, based on the information provided, it is logical to conclude that a deficiency of vitamin B12 can indeed incre